In [1]:
import datetime

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import scipy.stats

import seaborn as sns

from sklearn import cluster
from sklearn.metrics import silhouette_samples

# Importing and exploring the dataset:

In [2]:
!ls data

HibernationSurveyObservations.csv
HibernationSurveyObservationsCleaned.csv
RoostMetadata.txt
RoostObservations.csv


In [3]:
!file 'data/RoostObservations.csv'

data/RoostObservations.csv: ASCII text, with very long lines, with CRLF line terminators


In [4]:
!wc -l 'data/RoostObservations.csv'

23680 data/RoostObservations.csv


In [5]:
!head -n 3 'data/RoostObservations.csv'

observationID,recordKey,organisationName,datasetKey,surveyKey,sampleKey,gridReference,precision,siteKey,siteName,featureKey,startDate,endDate,dateType,recorder,determiner,pTaxonVersionKey,taxonName,authority,commonName,taxonGroup,sensitive,zeroAbundance,fullVersion,Abundance,Comments,HRGridReference,LocationName,RecordType,SpeciesIDMethod,SurveyType
463185100,7871,The Bat Conservation Trust,GA000616,1,1,NN5620,1km,,,129467,16/06/2007,16/06/2007,Day,,,NBNSYS0000005102,Plecotus auritus,"(Linnaeus, 1758)",Brown Long-eared Bat,terrestrial mammal,false,false,false,,,,,,,
463185128,1506,The Bat Conservation Trust,GA000616,1,1,NR2767,1km,,,130452,29/05/2010,29/05/2010,Day,,,NHMSYS0020001355,Pipistrellus pipistrellus sensu stricto,"(Schreber, 1774)",Common Pipistrelle,terrestrial mammal,false,false,false,,,,,,,


In [6]:
!tail -n 3 'data/RoostObservations.csv'

463209489,1739,The Bat Conservation Trust,GA000616,1,1,ST1821,1km,,,653453,30/06/2010,30/06/2010,Day,,,NHMSYS0000528030,Myotis mystacinus/brandtii,null,Whiskered/Brandt's Bat,terrestrial mammal,false,false,false,,,,,,,
463209975,21920,The Bat Conservation Trust,GA000616,1,1,SX8166,1km,,,665485,07/06/2004,07/06/2004,Day,,,NHMSYS0000080177,Rhinolophus hipposideros,"(Bechstein, 1800)",Lesser Horseshoe Bat,terrestrial mammal,false,false,false,,,,,,,
463209976,21921,The Bat Conservation Trust,GA000616,1,1,SX8166,1km,,,665485,17/06/2004,17/06/2004,Day,,,NHMSYS0000080177,Rhinolophus hipposideros,"(Bechstein, 1800)",Lesser Horseshoe Bat,terrestrial mammal,false,false,false,,,,,,,


In [7]:
pd.read_csv('data/RoostObservations.csv', nrows=5)

,observationID,recordKey,organisationName,datasetKey,surveyKey,sampleKey,gridReference,precision,siteKey,siteName,...,sensitive,zeroAbundance,fullVersion,Abundance,Comments,HRGridReference,LocationName,RecordType,SpeciesIDMethod,SurveyType
0,463185100,7871,The Bat Conservation Trust,GA000616,1,1,NN5620,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,463185128,1506,The Bat Conservation Trust,GA000616,1,1,NR2767,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,463185129,1507,The Bat Conservation Trust,GA000616,1,1,NR2767,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,463185205,756,The Bat Conservation Trust,GA000616,1,1,TA0333,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,463185255,12799,The Bat Conservation Trust,GA000616,1,1,NN7102,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Reading the data into a dataframe for further evaluation.

In [8]:
roost_df = pd.read_csv('data/RoostObservations.csv', engine='python')

In [9]:
roost_df.columns.tolist()

['observationID',
 'recordKey',
 'organisationName',
 'datasetKey',
 'surveyKey',
 'sampleKey',
 'gridReference',
 'precision',
 'siteKey',
 'siteName',
 'featureKey',
 'startDate',
 'endDate',
 'dateType',
 'recorder',
 'determiner',
 'pTaxonVersionKey',
 'taxonName',
 'authority',
 'commonName',
 'taxonGroup',
 'sensitive',
 'zeroAbundance',
 'fullVersion',
 'Abundance',
 'Comments',
 'HRGridReference',
 'LocationName',
 'RecordType',
 'SpeciesIDMethod',
 'SurveyType']

In [10]:
roost_df.dtypes

observationID         int64
recordKey             int64
organisationName     object
datasetKey           object
surveyKey             int64
sampleKey             int64
gridReference        object
precision            object
siteKey             float64
siteName            float64
featureKey            int64
startDate            object
endDate              object
dateType             object
recorder            float64
determiner          float64
pTaxonVersionKey     object
taxonName            object
authority            object
commonName           object
taxonGroup           object
sensitive              bool
zeroAbundance          bool
fullVersion            bool
Abundance           float64
Comments            float64
HRGridReference     float64
LocationName        float64
RecordType          float64
SpeciesIDMethod     float64
SurveyType          float64
dtype: object

In [11]:
roost_df.describe

<bound method NDFrame.describe of        observationID  recordKey            organisationName datasetKey  \
0          463185100       7871  The Bat Conservation Trust   GA000616   
1          463185128       1506  The Bat Conservation Trust   GA000616   
2          463185129       1507  The Bat Conservation Trust   GA000616   
3          463185205        756  The Bat Conservation Trust   GA000616   
4          463185255      12799  The Bat Conservation Trust   GA000616   
5          463185256      12800  The Bat Conservation Trust   GA000616   
6          463185451        690  The Bat Conservation Trust   GA000616   
7          463185492      17833  The Bat Conservation Trust   GA000616   
8          463185571       5151  The Bat Conservation Trust   GA000616   
9          463185572       5152  The Bat Conservation Trust   GA000616   
10         463185573       5153  The Bat Conservation Trust   GA000616   
11         463185574       5154  The Bat Conservation Trust   GA000616   
12  

In [12]:
len(roost_df['observationID'].unique())

23679

So there is one observationID value for each row.

In [13]:
len(roost_df['recordKey'].unique())
#one recordKey value for each row

23679

In [14]:
roost_df['organisationName'].unique()
#same value repeated, not useful

array(['The Bat Conservation Trust'], dtype=object)

In [15]:
roost_df['HRGridReference'].unique()

array([ nan])

In [16]:
roost_df['organisationName'].unique()
#same value repeated, not useful

array(['The Bat Conservation Trust'], dtype=object)

In [17]:
roost_df['datasetKey'].unique()
#same value repeated, not useful

array(['GA000616'], dtype=object)

In [18]:
roost_df['surveyKey'].unique()
#roost_df['surveyKey']
#same value repeated, not useful

array([1], dtype=int64)

In [19]:
roost_df['sampleKey'].unique()
#same value repeated, not useful

array([1], dtype=int64)

In [20]:
roost_df['gridReference'].unique()
#grid reference with 4 digit number

array(['NN5620', 'NR2767', 'TA0333', ..., 'SS4692', 'TA0067', 'SX8166'], dtype=object)

In [21]:
roost_df['precision'].unique()

array(['1km', '10km'], dtype=object)

# Discrepancies in the site precision values

Some values are for 1km others for 10km.

This is at odds with the metadata which states that the data resolution is 1km.

Since I am not considering location in my analysis this discrepancy is not likely to be a problem. Furthermore the number of sites with 10km precisions small so I have opted to leave the values as they are.

In [22]:
roost_df[roost_df['precision']=='10km']
#two sites appear to have a precision of 10km

,observationID,recordKey,organisationName,datasetKey,surveyKey,sampleKey,gridReference,precision,siteKey,siteName,...,sensitive,zeroAbundance,fullVersion,Abundance,Comments,HRGridReference,LocationName,RecordType,SpeciesIDMethod,SurveyType
121,463188032,8466,The Bat Conservation Trust,GA000616,1,1,ST76,10km,19786076.0,1030121.0,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,463188033,8467,The Bat Conservation Trust,GA000616,1,1,ST76,10km,19786076.0,1030121.0,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1074,463205666,7216,The Bat Conservation Trust,GA000616,1,1,SE30,10km,19785999.0,1080005.0,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075,463205667,7217,The Bat Conservation Trust,GA000616,1,1,SE30,10km,19785999.0,1080005.0,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,463205670,7235,The Bat Conservation Trust,GA000616,1,1,SE30,10km,19785999.0,1080005.0,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
len(roost_df['siteKey'].unique())
# Most values are NaN and those values that are not do not seem useful.

200

In [24]:
len(roost_df['siteName'].unique())
# Most values are NaN and those values that are not do not seem useful.

200

In [25]:
roost_df['featureKey'].unique()
len(roost_df['featureKey'].unique())
#roost_df['featureKey']

#It's not clear what this is but it also does not seem useful.

2026

According to the metadata the roosts are counted twice at sunset on two evenings during the survey period.

In [26]:
roost_df['startDate'].unique()
#roost_df['startDate']

array(['16/06/2007', '29/05/2010', '06/06/2010', ..., '27/07/2013',
       '13/07/2005', '28/05/2013'], dtype=object)

In [27]:
roost_df['endDate'].unique()

array(['16/06/2007', '29/05/2010', '06/06/2010', ..., '27/07/2013',
       '13/07/2005', '28/05/2013'], dtype=object)

In [28]:
roost_df['dateType'].unique()

array(['Day', 'Year'], dtype=object)

In [29]:
roost_df['recorder'].unique()
#Same value repeated, not useful.

array([ nan])

In [30]:
roost_df['determiner'].unique()
#Same value repeated, not useful.

array([ nan])

In [31]:
roost_df['pTaxonVersionKey'].unique()
#This seems to be the type of taxon directory used.
#Not really useful for my analysis.

array(['NBNSYS0000005102', 'NHMSYS0020001355', 'NHMSYS0020001356',
       'NBNSYS0100004720', 'NHMSYS0000080184', 'NHMSYS0000080177',
       'NHMSYS0000528008', 'NHMSYS0000080176', 'NHMSYS0000080186',
       'NHMSYS0000080183', 'NHMSYS0000376160', 'NHMSYS0000528028',
       'NHMSYS0000080185', 'NHMSYS0000528026', 'NHMSYS0000528030',
       'NBNSYS0000005103', 'NHMSYS0000528024'], dtype=object)

In [32]:
roost_df['taxonName'].unique()
#The bat species proper names.
len(roost_df['taxonName'].unique())
#Total of 17 bat species present.

17

In [33]:
roost_df['authority'].unique()
#Name of the person who first described the bat species and when.

array(['(Linnaeus, 1758)', '(Schreber, 1774)', '(Leach, 1825)',
       '(Kuhl, 1817)', '(Bechstein, 1800)', 'Blumenbach, 1779',
       '(Eversmann, 1845)', 'null', '(Fischer, 1829)'], dtype=object)

In [34]:
roost_df['commonName'].unique()
#The bat species common names.

array(['Brown Long-eared Bat', 'Common Pipistrelle', 'Pipistrelle',
       'Soprano Pipistrelle', "Natterer's Bat", 'Lesser Horseshoe Bat',
       'Serotine', 'Greater Horseshoe Bat', 'Noctule Bat', 'Whiskered Bat',
       'Bat', "Daubenton's Bat", 'Lesser Noctule', "Brandt's Bat",
       "Whiskered/Brandt's Bat", 'Grey Long-eared Bat', "Bechstein's Bat"], dtype=object)

In [35]:
roost_df['taxonGroup'].unique()
#Same value repeated, so not useful.

array(['terrestrial mammal'], dtype=object)

In [36]:
roost_df['sensitive'].unique()
#Same value repeated, so not useful.

array([False], dtype=object)

In [37]:
roost_df['zeroAbundance'].unique()
#Same value repeated, so not useful.

array([False], dtype=object)

In [38]:
roost_df['fullVersion'].unique()
#Same value repeated, so not useful.

array([False], dtype=object)

The above just indicates that this is not the full dataset. 

Some entries are restricted for legal and privacy reasons as stated in the metadata.

In [39]:
roost_df['Abundance'].unique()
#Same value repeated, so not useful.

array([ nan])

In [40]:
roost_df['Comments'].unique()
#Same value NaN repeated, so not useful.

array([ nan])

In [41]:
roost_df['HRGridReference'].unique()
#Same value repeated, so not useful.
#Probaby high resolution grid reference removed due to sensitivity.

array([ nan])

In [42]:
roost_df['LocationName'].unique()
#Same value repeated, so not useful.

array([ nan])

In [43]:
roost_df['RecordType'].unique()
#Same value repeated, so not useful.

array([ nan])

In [44]:
roost_df['SpeciesIDMethod'].unique()
#Same value repeated, so not useful.

array([ nan])

In [45]:
roost_df['SurveyType'].unique()
#Same value repeated, so not useful.

array([ nan])

# Shaping the table into a more useful form:

In [46]:
roost_df['startDate'].head()

0    16/06/2007
1    29/05/2010
2    06/06/2010
3    13/07/2010
4    11/07/2010
Name: startDate, dtype: object

In [47]:
roost_df['endDate'].head()

0    16/06/2007
1    29/05/2010
2    06/06/2010
3    13/07/2010
4    11/07/2010
Name: endDate, dtype: object

In [48]:
roost_df.head()

,observationID,recordKey,organisationName,datasetKey,surveyKey,sampleKey,gridReference,precision,siteKey,siteName,...,sensitive,zeroAbundance,fullVersion,Abundance,Comments,HRGridReference,LocationName,RecordType,SpeciesIDMethod,SurveyType
0,463185100,7871,The Bat Conservation Trust,GA000616,1,1,NN5620,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,463185128,1506,The Bat Conservation Trust,GA000616,1,1,NR2767,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,463185129,1507,The Bat Conservation Trust,GA000616,1,1,NR2767,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,463185205,756,The Bat Conservation Trust,GA000616,1,1,TA0333,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,463185255,12799,The Bat Conservation Trust,GA000616,1,1,NN7102,1km,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Converting the data types that represent date and time and adding a year column:

In [49]:
roost_df['startDate'] = pd.to_datetime(roost_df['startDate'])
roost_df['endDate'] = pd.to_datetime(roost_df['endDate'])
roost_df['year'] = roost_df['endDate'].dt.year

In [50]:
roost_df.head()

columns = roost_df.columns.tolist()
columns = ['observationID', 'gridReference', 'precision', 
          'startDate', 'endDate', 'year', 'taxonName']
columns

roost_short_df = roost_df[columns]
roost_short_df.tail(5)

,observationID,gridReference,precision,startDate,endDate,year,taxonName
23674,463209487,ST1821,1km,2011-06-15,2011-06-15,2011,Myotis mystacinus/brandtii
23675,463209488,ST1821,1km,2010-06-29,2010-06-29,2010,Myotis mystacinus/brandtii
23676,463209489,ST1821,1km,2010-06-30,2010-06-30,2010,Myotis mystacinus/brandtii
23677,463209975,SX8166,1km,2004-07-06,2004-07-06,2004,Rhinolophus hipposideros
23678,463209976,SX8166,1km,2004-06-17,2004-06-17,2004,Rhinolophus hipposideros


According to the metadata the roosts are counted twice at sunset on two evenings during the survey period. There does not seem to be a pattern in when the second observation is made.

It seems likely then that at least some bats are counted twice but this assumption also rest upon the idea that bats do not move roosting sites frequently. In any case there is obviously no way to determining or accounting for this.

Another confounding factor is that observations are made sporadically so it’s possible that there are local and temporal effects on the data as a result. But again there is no way to account for this from the data.

Given that the start and end dates are the same I have decided to use the end date value and a derived year value from end date as the time values for exploring the dataset.

There is also no clear way of differentiating one site from another except for the grid reference column so I will take grid reference to be synonymous to site for my analysis.

In [51]:
roost_short_df[roost_short_df['gridReference']=='SX8166'].sort_values(by='startDate')

,observationID,gridReference,precision,startDate,endDate,year,taxonName
23616,463204958,SX8166,1km,2000-05-06,2000-05-06,2000,Rhinolophus hipposideros
23617,463204959,SX8166,1km,2000-08-06,2000-08-06,2000,Rhinolophus hipposideros
23618,463204960,SX8166,1km,2002-04-06,2002-04-06,2002,Rhinolophus hipposideros
23619,463204961,SX8166,1km,2002-06-18,2002-06-18,2002,Rhinolophus hipposideros
23621,463204963,SX8166,1km,2003-06-17,2003-06-17,2003,Rhinolophus hipposideros
23620,463204962,SX8166,1km,2003-07-06,2003-07-06,2003,Rhinolophus hipposideros
23678,463209976,SX8166,1km,2004-06-17,2004-06-17,2004,Rhinolophus hipposideros
23677,463209975,SX8166,1km,2004-07-06,2004-07-06,2004,Rhinolophus hipposideros
23629,463205002,SX8166,1km,2005-05-31,2005-05-31,2005,Rhinolophus hipposideros
23630,463205003,SX8166,1km,2005-06-16,2005-06-16,2005,Rhinolophus hipposideros


Removing the start dates.

In [52]:
roost_df.head()

columns = roost_df.columns.tolist()
columns = ['observationID', 'gridReference', 'precision', 'endDate',
          'year', 'taxonName']
columns

roost_short2_df = roost_df[columns]
roost_short2_df.tail(5)

,observationID,gridReference,precision,endDate,year,taxonName
23674,463209487,ST1821,1km,2011-06-15,2011,Myotis mystacinus/brandtii
23675,463209488,ST1821,1km,2010-06-29,2010,Myotis mystacinus/brandtii
23676,463209489,ST1821,1km,2010-06-30,2010,Myotis mystacinus/brandtii
23677,463209975,SX8166,1km,2004-07-06,2004,Rhinolophus hipposideros
23678,463209976,SX8166,1km,2004-06-17,2004,Rhinolophus hipposideros


# Investigating the data further

It seems like some sites are not surveyed very often.

In [53]:
roost_short2_df.sort_values(by='gridReference', ascending=False).head(10)

,observationID,gridReference,precision,endDate,year,taxonName
9413,463210026,TR3358,1km,2003-12-31,2003,Myotis nattereri
9420,463210038,TR3358,1km,2005-07-06,2005,Myotis nattereri
9421,463210039,TR3358,1km,2005-06-21,2005,Myotis nattereri
12198,463195070,TR2666,1km,1999-08-06,1999,Pipistrellus pipistrellus sensu stricto
12199,463195071,TR2666,1km,1999-06-25,1999,Pipistrellus pipistrellus sensu stricto
12197,463195069,TR2666,1km,1998-06-24,1998,Pipistrellus pipistrellus sensu stricto
12196,463195068,TR2666,1km,1998-06-15,1998,Pipistrellus pipistrellus sensu stricto
12195,463195067,TR2666,1km,1997-06-20,1997,Pipistrellus pipistrellus sensu stricto
12194,463195066,TR2666,1km,1997-11-06,1997,Pipistrellus pipistrellus sensu stricto
12200,463195072,TR2666,1km,2003-11-06,2003,Pipistrellus pipistrellus sensu stricto


The survey data is from 1977 to 2014.

In [54]:
roost_short2_df.sort_values(by='year').head()
#earliest year 1977

roost_short2_df.sort_values(by='year', ascending=False).head()
#latest 2014

,observationID,gridReference,precision,endDate,year,taxonName
19260,463195074,SD4983,1km,2014-07-06,2014,Myotis nattereri
14768,463204568,NY6423,1km,2014-06-23,2014,Pipistrellus pygmaeus
21922,463204553,SE1546,1km,2014-06-15,2014,Plecotus auritus
14769,463204569,NY6423,1km,2014-07-22,2014,Pipistrellus pygmaeus
1156,463207039,NC0825,1km,2014-06-27,2014,Pipistrellus pipistrellus sensu stricto


There are 23,679 bats recorded in the data.

In [55]:
len(roost_short_df.observationID.unique())
#23679 unique observations

23679

There are 2,026 unique sites.

In [56]:
len(roost_short_df.gridReference.unique())
#2026 unique sites.

2026

# Investigating bat counts by gridReference over time

In [57]:
roost_short_grid_count_df = roost_short_df.groupby(['gridReference', 'year', 'endDate', 'taxonName'])['taxonName'].count()

In [58]:
rs_unstack = roost_short_grid_count_df.unstack()

In [59]:
rs_unstack.head()

taxonName                      Chiroptera  Eptesicus serotinus  \
gridReference year endDate                                       
C5010         2005 2005-06-23         NaN                  NaN   
                   2005-10-06         NaN                  NaN   
D1240         1998 1998-06-14         NaN                  NaN   
                   1998-06-25         NaN                  NaN   
              1999 1999-06-21         NaN                  NaN   

taxonName                      Myotis bechsteinii  Myotis brandtii  \
gridReference year endDate                                           
C5010         2005 2005-06-23                 NaN              NaN   
                   2005-10-06                 NaN              NaN   
D1240         1998 1998-06-14                 NaN              NaN   
                   1998-06-25                 NaN              NaN   
              1999 1999-06-21                 NaN              NaN   

taxonName                      Myotis daubentonii  Myotis mystacinus  \
gridReference year endDate                                             
C5010         2005 2005-06-23                 NaN                NaN   
                   2005-10-06                 NaN                NaN   
D1240         1998 1998-06-14                 NaN                NaN   
                   1998-06-25                 NaN                NaN   
              1999 1999-06-21                 NaN                NaN   

taxonName                      Myotis mystacinus/brandtii  Myotis nattereri  \
gridReference year endDate                                                    
C5010         2005 2005-06-23                         NaN               NaN   
                   2005-10-06                         NaN               NaN   
D1240         1998 1998-06-14                         NaN               NaN   
                   1998-06-25                         NaN               NaN   
              1999 1999-06-21                         NaN               NaN   

taxonName                      Nyctalus leisleri  Nyctalus noctula  \
gridReference year endDate                                           
C5010         2005 2005-06-23                NaN               NaN   
                   2005-10-06                NaN               NaN   
D1240         1998 1998-06-14                NaN               NaN   
                   1998-06-25                NaN               NaN   
              1999 1999-06-21                NaN               NaN   

taxonName                      Pipistrellus pipistrellus sensu lato  \
gridReference year endDate                                            
C5010         2005 2005-06-23                                   1.0   
                   2005-10-06                                   1.0   
D1240         1998 1998-06-14                                   NaN   
                   1998-06-25                                   NaN   
              1999 1999-06-21                                   NaN   

taxonName                      Pipistrellus pipistrellus sensu stricto  \
gridReference year endDate                                               
C5010         2005 2005-06-23                                      NaN   
                   2005-10-06                                      NaN   
D1240         1998 1998-06-14                                      1.0   
                   1998-06-25                                      1.0   
              1999 1999-06-21                                      1.0   

taxonName                      Pipistrellus pygmaeus  Plecotus auritus  \
gridReference year endDate                                               
C5010         2005 2005-06-23                    NaN               NaN   
                   2005-10-06                    NaN               NaN   
D1240         1998 1998-06-14                    NaN               NaN   
                   1998-06-25                    NaN               NaN   
              1999 1999-06-21      

# Filling NaN values


In [60]:
rs_unstack.fillna(0, inplace=True)

In [61]:
rs_unstack.head()

taxonName                      Chiroptera  Eptesicus serotinus  \
gridReference year endDate                                       
C5010         2005 2005-06-23         0.0                  0.0   
                   2005-10-06         0.0                  0.0   
D1240         1998 1998-06-14         0.0                  0.0   
                   1998-06-25         0.0                  0.0   
              1999 1999-06-21         0.0                  0.0   

taxonName                      Myotis bechsteinii  Myotis brandtii  \
gridReference year endDate                                           
C5010         2005 2005-06-23                 0.0              0.0   
                   2005-10-06                 0.0              0.0   
D1240         1998 1998-06-14                 0.0              0.0   
                   1998-06-25                 0.0              0.0   
              1999 1999-06-21                 0.0              0.0   

taxonName                      Myotis daubentonii  Myotis mystacinus  \
gridReference year endDate                                             
C5010         2005 2005-06-23                 0.0                0.0   
                   2005-10-06                 0.0                0.0   
D1240         1998 1998-06-14                 0.0                0.0   
                   1998-06-25                 0.0                0.0   
              1999 1999-06-21                 0.0                0.0   

taxonName                      Myotis mystacinus/brandtii  Myotis nattereri  \
gridReference year endDate                                                    
C5010         2005 2005-06-23                         0.0               0.0   
                   2005-10-06                         0.0               0.0   
D1240         1998 1998-06-14                         0.0               0.0   
                   1998-06-25                         0.0               0.0   
              1999 1999-06-21                         0.0               0.0   

taxonName                      Nyctalus leisleri  Nyctalus noctula  \
gridReference year endDate                                           
C5010         2005 2005-06-23                0.0               0.0   
                   2005-10-06                0.0               0.0   
D1240         1998 1998-06-14                0.0               0.0   
                   1998-06-25                0.0               0.0   
              1999 1999-06-21                0.0               0.0   

taxonName                      Pipistrellus pipistrellus sensu lato  \
gridReference year endDate                                            
C5010         2005 2005-06-23                                   1.0   
                   2005-10-06                                   1.0   
D1240         1998 1998-06-14                                   0.0   
                   1998-06-25                                   0.0   
              1999 1999-06-21                                   0.0   

taxonName                      Pipistrellus pipistrellus sensu stricto  \
gridReference year endDate                                               
C5010         2005 2005-06-23                                      0.0   
                   2005-10-06                                      0.0   
D1240         1998 1998-06-14                                      1.0   
                   1998-06-25                                      1.0   
              1999 1999-06-21                                      1.0   

taxonName                      Pipistrellus pygmaeus  Plecotus auritus  \
gridReference year endDate                                               
C5010         2005 2005-06-23                    0.0               0.0   
                   2005-10-06                    0.0               0.0   
D1240         1998 1998-06-14                    0.0               0.0   
                   1998-06-25                    0.0               0.0   
              1999 1999-06-21      

Looking at the above data it seems that most sites contain only one bat species.

Sites with multiple species abundance at the same time do exist however for example gridReference H1861 has several species present at the same time in 2004, 2008 and 2009.

In [62]:
roost_short_df[roost_short_df['gridReference']=='H1861'].sort_values(by='endDate')

,observationID,gridReference,precision,startDate,endDate,year,taxonName
20071,463198129,H1861,1km,1998-06-06,1998-06-06,1998,Pipistrellus pygmaeus
20072,463198130,H1861,1km,1998-06-15,1998-06-15,1998,Pipistrellus pygmaeus
20073,463198131,H1861,1km,1998-06-25,1998-06-25,1998,Pipistrellus pygmaeus
20076,463198134,H1861,1km,1999-06-20,1999-06-20,1999,Pipistrellus pygmaeus
20075,463198133,H1861,1km,1999-06-25,1999-06-25,1999,Pipistrellus pygmaeus
20074,463198132,H1861,1km,1999-12-06,1999-12-06,1999,Pipistrellus pygmaeus
20077,463198135,H1861,1km,2000-06-14,2000-06-14,2000,Pipistrellus pygmaeus
20078,463198136,H1861,1km,2000-06-21,2000-06-21,2000,Pipistrellus pygmaeus
20080,463198138,H1861,1km,2001-06-23,2001-06-23,2001,Pipistrellus pygmaeus
20079,463198137,H1861,1km,2001-12-06,2001-12-06,2001,Pipistrellus pygmaeus


# Exploring bat co-habitation

Perhaps there is a pattern to the above grouping that could indicate if certain bat species are more likely to be found with another species of bat.

I will explore this possibility by deriving statistical data from the dataset and applying the K-means clustering algorithm to try to determine which species tend to group together.

I have chosen K-means because it can be used to group species based on similar properties. By deriving statistical data from the dataset and applying K-means. I will explore if the statistical properties of a particular species indicate whether or not it is likely to be found in conjunction with other bat species.

# Examining statistical values for the bat counts grouped by gridReference over time

In [63]:
rs_unstack.describe()

taxonName,Chiroptera,Eptesicus serotinus,Myotis bechsteinii,Myotis brandtii,Myotis daubentonii,Myotis mystacinus,Myotis mystacinus/brandtii,Myotis nattereri,Nyctalus leisleri,Nyctalus noctula,Pipistrellus pipistrellus sensu lato,Pipistrellus pipistrellus sensu stricto,Pipistrellus pygmaeus,Plecotus auritus,Plecotus austriacus,Rhinolophus ferrumequinum,Rhinolophus hipposideros
count,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000,22858.000000
mean,0.002362,0.048429,0.000962,0.002625,0.007000,0.004244,0.001444,0.045498,0.000481,0.003150,0.152157,0.283271,0.193411,0.084609,0.000219,0.022487,0.183568
std,0.048548,0.215490,0.031009,0.051168,0.093279,0.065006,0.037969,0.209237,0.021932,0.056037,0.360882,0.454175,0.409662,0.279560,0.014789,0.148263,0.396630
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,9.000000,2.000000,1.000000,1.000000,3.000000


A negative effect of grouping in this way is that the bat count value is for gridReferences over time where observations were taken rather than the total number of bats for each species observed.

The total number of bats observed for each species can be extracted using the sum function however.

In [64]:
rs_unstack.sum()

taxonName
Chiroptera                                   54.0
Eptesicus serotinus                        1107.0
Myotis bechsteinii                           22.0
Myotis brandtii                              60.0
Myotis daubentonii                          160.0
Myotis mystacinus                            97.0
Myotis mystacinus/brandtii                   33.0
Myotis nattereri                           1040.0
Nyctalus leisleri                            11.0
Nyctalus noctula                             72.0
Pipistrellus pipistrellus sensu lato       3478.0
Pipistrellus pipistrellus sensu stricto    6475.0
Pipistrellus pygmaeus                      4421.0
Plecotus auritus                           1934.0
Plecotus austriacus                           5.0
Rhinolophus ferrumequinum                   514.0
Rhinolophus hipposideros                   4196.0
dtype: float64

# Shaping data to look at its properties

Putting the data into a more readable and useful form for clustering by extracting statistical values.

In [65]:
df4 = pd.DataFrame({'mean': rs_unstack.mean(), 'sum': rs_unstack.sum(),
                   'max': rs_unstack.max(), 'standard_deviation': rs_unstack.std(),
                   'min': rs_unstack.min()})
df4

,max,mean,min,standard_deviation,sum
taxonName,,,,,
Chiroptera,1.0,0.002362,0.0,0.048548,54.0
Eptesicus serotinus,2.0,0.048429,0.0,0.215490,1107.0
Myotis bechsteinii,1.0,0.000962,0.0,0.031009,22.0
Myotis brandtii,1.0,0.002625,0.0,0.051168,60.0
Myotis daubentonii,2.0,0.007000,0.0,0.093279,160.0
Myotis mystacinus,1.0,0.004244,0.0,0.065006,97.0
Myotis mystacinus/brandtii,1.0,0.001444,0.0,0.037969,33.0
Myotis nattereri,2.0,0.045498,0.0,0.209237,1040.0
Nyctalus leisleri,1.0,0.000481,0.0,0.021932,11.0


The mean is the average number of a particular species at a particular site over time.

The standard deviation here is saying how spread out the species count over all the sites over time is from the mean.

The max is the highest number of bats recorded at a particular site over time and minimum values indicate that no site has every species of bat present.

# Choosing values for clustering

The sum of each bat species against standard deviation for that species should give a good representation of the data for clustering.  

This rests on the assumption that the most abundant bat species have a higher probability of co-habitation, and that the higher the standard deviation the more spread out the values for a particular species.  This should also translate to a higher chance of co-habitation.

In [66]:
plt.scatter(df4['sum'],
            df4['standard_deviation'])

plt.xlabel('sum')
plt.ylabel('standard_deviation')

plt.title('Bat species properties max by standard deviation in roost data.')

# Getting the assigned clusters using k-means++

In [67]:
clusteringData_df = df4[['sum', 'standard_deviation']]
#clusteringData_df

In [68]:
#Adding this because the import doesn't seem to 'stick'.
#That is I had problems with it not recognising the import from above after some time.
#Adding the import statment before using it seems to work ok.
from sklearn import cluster

#getting kmeans values using kmeans++
kmeans2 = cluster.KMeans(n_clusters=2)

# Then call the fit function on the clusterer to fit the data to the 3 means.
kmeans2_asigned_clusters = kmeans2.fit(clusteringData_df)

# And see how the data points have been assigned to clusters:
kmeans2_asigned_clusters.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1])

# Plotting the assigned clusters

In [69]:
#to make the plot more readable
plt.figure(figsize=(10, 8))

# Plot the data points which in the cluster labelled '0'
plt.scatter(clusteringData_df['sum'][kmeans2_asigned_clusters.labels_==0],
            clusteringData_df['standard_deviation'][kmeans2_asigned_clusters.labels_==0],
            color='red', marker='o', label='cluster 0')

# Plot the data points which in the cluster labelled '1'
plt.scatter(clusteringData_df['sum'][kmeans2_asigned_clusters.labels_==1],
            clusteringData_df['standard_deviation'][kmeans2_asigned_clusters.labels_==1],
            color='blue', marker='o', label='cluster 1')

plt.legend()

plt.xlabel('max')
plt.ylabel('standard_deviation')

#remember the change the name
plt.title('2 K-means clustering')

#plt.ylim(-1,10)
#plt.xlim(-1, 400)

#Plot centroids
#gotta change the name here also
for (cx, cy) in kmeans2_asigned_clusters.cluster_centers_:
    plt.plot(cx, cy, color='black', marker='x', mew=2)

# Silhouette analysis of the K-means2 clustering

In [70]:
kmeans2_asigned_clusters.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1])

# Adding the cluster values calculated above to a new dataframe

In [71]:
k_means2_label = kmeans2_asigned_clusters.labels_

sil_kmeans2 = pd.DataFrame({'standard_deviation': clusteringData_df['standard_deviation'],
                   'sum': clusteringData_df['sum'],
                   'cluster': k_means2_label})
sil_kmeans2

,cluster,standard_deviation,sum
taxonName,,,
Chiroptera,0,0.048548,54.0
Eptesicus serotinus,0,0.215490,1107.0
Myotis bechsteinii,0,0.031009,22.0
Myotis brandtii,0,0.051168,60.0
Myotis daubentonii,0,0.093279,160.0
Myotis mystacinus,0,0.065006,97.0
Myotis mystacinus/brandtii,0,0.037969,33.0
Myotis nattereri,0,0.209237,1040.0
Nyctalus leisleri,0,0.021932,11.0


# Adding the silhouette values for the clustering of K-means 2

In [72]:
sil_kmeans2['silhouette'] = silhouette_samples(sil_kmeans2[['sum', 'standard_deviation']], np.array(sil_kmeans2['cluster']))

sil_kmeans2

,cluster,standard_deviation,sum,silhouette
taxonName,,,,
Chiroptera,0,0.048548,54.0,0.914696
Eptesicus serotinus,0,0.215490,1107.0,0.742234
Myotis bechsteinii,0,0.031009,22.0,0.912004
Myotis brandtii,0,0.051168,60.0,0.914912
Myotis daubentonii,0,0.093279,160.0,0.909258
Myotis mystacinus,0,0.065006,97.0,0.913981
Myotis mystacinus/brandtii,0,0.037969,33.0,0.913187
Myotis nattereri,0,0.209237,1040.0,0.760976
Nyctalus leisleri,0,0.021932,11.0,0.910432


# Defining the colour map for the silouhette

In [73]:
colourMap_dict = {0:'red',
                  1:'blue'}

colourMap_dict

{0: 'red', 1: 'blue'}

# Plotting the silhouette

In [74]:
sil = sil_kmeans2.sort_values(['cluster', 'silhouette'])
sil.index = list(range(len(sil_kmeans2)))

plt.figure(figsize=(10, 8))

for cluster in set(sil['cluster']):
    plt.bar(sil[sil['cluster']==cluster].index,
         sil[sil['cluster']==cluster]['silhouette'],
         color = colourMap_dict[cluster], label='Cluster ' + str(cluster))

    
plt.title('Silhouette plot of 2 K-means clustering')

plt.legend()

plt.xlabel('Number of data point')
plt.ylabel('Silhouette coefficient')

plt.ylim(0, 1.2)
plt.xlim(0,17)

(0, 17)

For both clusters there is a high level of variation.

One cluster is clearly dominating the data points but with a high variation of values with the low end being ~0.39 to a high end of ~0.86.

The range of values in the smaller cluster is also quite wide from ~0.5 to ~0.91. 

The lower values in the clusters indicate that they may be further form the mean of their own clusters; they may also be close to the members of the nearest cluster or both.

So it seems likely that increasing the value for K can increasing the number of clusters may give a better clustering.

# K-means3

In [75]:
#Adding this because the import doesn't seem to 'stick'.
#That is I had problems with it not recognising the import from above after some time.
#Adding the import statment before using it seems to work ok.
from sklearn import cluster

#getting kmeans values using kmeans++
kmeans3 = cluster.KMeans(n_clusters=3)

# Then call the fit function on the clusterer to fit the data to the 3 means.
kmeans3_asigned_clusters = kmeans3.fit(clusteringData_df)

# And see how the data points have been assigned to clusters:
kmeans3_asigned_clusters.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 1])

# Plot for K-means 3

In [76]:
#to make the plot more readable
plt.figure(figsize=(10, 8))

# Plot the data points which in the cluster labelled '0'
plt.scatter(clusteringData_df['sum'][kmeans3_asigned_clusters.labels_==0],
            clusteringData_df['standard_deviation'][kmeans3_asigned_clusters.labels_==0],
            color='red', marker='o', label='cluster 0')

# Plot the data points which in the cluster labelled '1'
plt.scatter(clusteringData_df['sum'][kmeans3_asigned_clusters.labels_==1],
            clusteringData_df['standard_deviation'][kmeans3_asigned_clusters.labels_==1],
            color='blue', marker='o', label='cluster 1')

# Plot the data points which in the cluster labelled '2'
plt.scatter(clusteringData_df['sum'][kmeans3_asigned_clusters.labels_==2],
            clusteringData_df['standard_deviation'][kmeans3_asigned_clusters.labels_==2],
            color='green', marker='o', label='cluster 2')

plt.legend()

plt.xlabel('sum')
plt.ylabel('standard_deviation')

#remember to change the name
plt.title('3-means clustering')

#plt.ylim(-1,10)
plt.xlim(-1000, 9000)

#Plot centroids
#gotta change the name here also
for (cx, cy) in kmeans3_asigned_clusters.cluster_centers_:
    plt.plot(cx, cy, color='black', marker='x', mew=2)

# Data for silhouette of 3 K-means

In [77]:
k_means3_label = kmeans3_asigned_clusters.labels_

sil_kmeans3 = pd.DataFrame({'standard_deviation': clusteringData_df['standard_deviation'],
                   'sum': clusteringData_df['sum'],
                   'cluster': k_means3_label})
#sil_kmeans3

# Silhouette values for K-means 3

In [78]:
sil_kmeans3['silhouette'] = silhouette_samples(sil_kmeans3[['sum', 'standard_deviation']], np.array(sil_kmeans3['cluster']))

#sil_kmeans3

In [79]:
colourMap_3kmeans_dict = {0:'red', 1:'blue', 2:'green'}

colourMap_3kmeans_dict

{0: 'red', 1: 'blue', 2: 'green'}

In [80]:
sil_kmeans3 = sil_kmeans3.sort_values(['cluster', 'silhouette'])
sil_kmeans3.index = list(range(len(sil)))

plt.figure(figsize=(10, 8))

for cluster in set(sil_kmeans3['cluster']):
    plt.bar(sil_kmeans3[sil_kmeans3['cluster']==cluster].index,
         sil_kmeans3[sil_kmeans3['cluster']==cluster]['silhouette'],
         color = colourMap_3kmeans_dict[cluster], label='Cluster ' + str(cluster))

    
plt.title('Silhouette plot for K-means 3')

plt.legend()

plt.xlabel('Number of data point')
plt.ylabel('Silhouette coefficient')

plt.ylim(-0.2, 1.2)
plt.xlim(0,17)

(0, 17)

The three clusters are clearly of a different size. The two smaller clusters have much less variation in the range of values they encompass especially the smaller of the two which encompasses a single data point. 

The larger cluster again dominates the others and has an even greater range of values.

Because of the large amount of variation in the larger cluster increasing the clusters will be examined to determine if K=4 is a better fit for the data.


# k-means 4

In [81]:
#Adding this because the import doesn't seem to 'stick'.
from sklearn import cluster

# First, create an instance of the clusterer with k=3
kmeans4 = cluster.KMeans(n_clusters=4)

# Then call the fit function on the clusterer to fit the data to the 3 means.
kmeans4_asigned_clusters = kmeans4.fit(clusteringData_df)

# And see how the data points have been assigned to clusters:
kmeans4_asigned_clusters.labels_

array([0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 1, 3, 0, 0, 1])

In [82]:
#to make the plot more readable
plt.figure(figsize=(10, 8))

# Plot the data points which in the cluster labelled '0'
plt.scatter(clusteringData_df['sum'][kmeans4_asigned_clusters.labels_==0],
            clusteringData_df['standard_deviation'][kmeans4_asigned_clusters.labels_==0],
            color='red', marker='o', label='cluster 0')

# Plot the data points which in the cluster labelled '1'
plt.scatter(clusteringData_df['sum'][kmeans4_asigned_clusters.labels_==1],
            clusteringData_df['standard_deviation'][kmeans4_asigned_clusters.labels_==1],
            color='blue', marker='o', label='cluster 1')

# Plot the data points which in the cluster labelled '2'
plt.scatter(clusteringData_df['sum'][kmeans4_asigned_clusters.labels_==2],
            clusteringData_df['standard_deviation'][kmeans4_asigned_clusters.labels_==2],
            color='green', marker='o', label='cluster 2')

# Plot the data points which in the cluster labelled '3'
plt.scatter(clusteringData_df['sum'][kmeans4_asigned_clusters.labels_==3],
            clusteringData_df['standard_deviation'][kmeans4_asigned_clusters.labels_==3],
            color='orange', marker='o', label='cluster 3')

plt.legend()

plt.xlabel('sum')
plt.ylabel('standard_deviation')

#remember to change the name
plt.title('4 K-means clustering')

#plt.ylim(-1,10)
plt.xlim(-1000, 9000)

#Plot centroids
#gotta change the name here also
for (cx, cy) in kmeans4_asigned_clusters.cluster_centers_:
    plt.plot(cx, cy, color='black', marker='x', mew=2)

In [83]:
colourMap_4kmeans_dict = {0:'red', 1:'blue', 2:'green', 3:'purple'}

colourMap_4kmeans_dict

{0: 'red', 1: 'blue', 2: 'green', 3: 'purple'}

In [84]:
k_means4_label = kmeans4_asigned_clusters.labels_

sil_kmeans4 = pd.DataFrame({'standard_deviation': clusteringData_df['standard_deviation'],
                   'sum': clusteringData_df['sum'],
                   'cluster': k_means4_label})
#sil_kmeans4.sort_values(by='cluster')

sil_kmeans4['silhouette'] = silhouette_samples(sil_kmeans4[['sum', 'standard_deviation']], np.array(sil_kmeans4['cluster']))
#for some reason just using the variable sil_kmeans4 leads to the index being dropped in later use.
needed_for_later = sil_kmeans4#.sort_values(by='cluster')

In [85]:
sil_kmeans4 = sil_kmeans4.sort_values(['cluster', 'silhouette'])
sil_kmeans4.index = list(range(len(sil)))

plt.figure(figsize=(10, 8))

for cluster in set(sil_kmeans4['cluster']):
    plt.bar(sil_kmeans4[sil_kmeans4['cluster']==cluster].index,
         sil_kmeans4[sil_kmeans4['cluster']==cluster]['silhouette'],
         color = colourMap_4kmeans_dict[cluster], label='Cluster ' + str(cluster))

    
plt.title('Silhouette plot of 4 K-means clustering')

plt.legend()

plt.xlabel('Number of data point')
plt.ylabel('Silhouette coefficient')

plt.ylim(-0.2, 1.5)
plt.xlim(0,17)

(0, 17)

There is considerably less variation in the largest of the clusters when compared with k=3. There is still however a single data point with a considerable different in silhouette coefficient to the other points in the same cluster. 

The next two largest clusters contain the same number of data point but one is showing greater cohesion then the other.

The smallest cluster of a single data point remains indicating that this may be an outlier.

Considering there is still a lot of variation k = 5 will be considered.

# k-means 5

In [86]:
#Adding this because the import doesn't seem to 'stick'.
from sklearn import cluster

# First, create an instance of the clusterer with k=3
kmeans5 = cluster.KMeans(n_clusters=5)

# Then call the fit function on the clusterer to fit the data to the 3 means.
kmeans5_asigned_clusters = kmeans5.fit(clusteringData_df)

# And see how the data points have been assigned to clusters:
kmeans5_asigned_clusters.labels_

array([0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 2, 1, 2, 3, 0, 0, 2])

In [87]:
#to make the plot more readable
plt.figure(figsize=(10, 8))

# Plot the data points which in the cluster labelled '0'
plt.scatter(clusteringData_df['sum'][kmeans5_asigned_clusters.labels_==0],
            clusteringData_df['standard_deviation'][kmeans5_asigned_clusters.labels_==0],
            color='red', marker='o', label='cluster 0')

# Plot the data points which in the cluster labelled '1'
plt.scatter(clusteringData_df['sum'][kmeans5_asigned_clusters.labels_==1],
            clusteringData_df['standard_deviation'][kmeans5_asigned_clusters.labels_==1],
            color='blue', marker='o', label='cluster 1')

# Plot the data points which in the cluster labelled '2'
plt.scatter(clusteringData_df['sum'][kmeans5_asigned_clusters.labels_==2],
            clusteringData_df['standard_deviation'][kmeans5_asigned_clusters.labels_==2],
            color='green', marker='o', label='cluster 2')

# Plot the data points which in the cluster labelled '3'
plt.scatter(clusteringData_df['sum'][kmeans5_asigned_clusters.labels_==3],
            clusteringData_df['standard_deviation'][kmeans5_asigned_clusters.labels_==3],
            color='purple', marker='o', label='cluster 3')

# Plot the data points which in the cluster labelled '4'
plt.scatter(clusteringData_df['sum'][kmeans5_asigned_clusters.labels_==4],
            clusteringData_df['standard_deviation'][kmeans5_asigned_clusters.labels_==4],
            color='orange', marker='o', label='cluster 4')

plt.legend()

plt.ylabel('standard_deviation')
plt.xlabel('sum')

#remember the change the name
plt.title('5 K-means Clustering')

#plt.ylim(-1,10)
plt.xlim(-1000, 9000)

#Plot centroids
#gotta change the name here also
for (cx, cy) in kmeans5_asigned_clusters.cluster_centers_:
    plt.plot(cx, cy, color='black', marker='x', mew=2)

In [88]:
colourMap_5kmeans_dict = {0:'red', 1:'blue', 2:'green', 3:'purple', 4:'orange'}

colourMap_5kmeans_dict

{0: 'red', 1: 'blue', 2: 'green', 3: 'purple', 4: 'orange'}

In [89]:
k_means5_label = kmeans5_asigned_clusters.labels_


sil_kmeans5 = pd.DataFrame({'standard_deviation': clusteringData_df['standard_deviation'],
                   'sum': clusteringData_df['sum'],
                   'cluster': k_means5_label})

#sil_kmeans5

In [90]:
sil_kmeans5['silhouette'] = silhouette_samples(sil_kmeans5[['sum', 'standard_deviation']], np.array(sil_kmeans4['cluster']))

sil_kmeans5

,cluster,standard_deviation,sum,silhouette
taxonName,,,,
Chiroptera,0,0.048548,54.0,0.834588
Eptesicus serotinus,4,0.215490,1107.0,-0.115403
Myotis bechsteinii,0,0.031009,22.0,0.825041
Myotis brandtii,0,0.051168,60.0,0.835235
Myotis daubentonii,0,0.093279,160.0,0.809461
Myotis mystacinus,0,0.065006,97.0,0.830295
Myotis mystacinus/brandtii,0,0.037969,33.0,0.829338
Myotis nattereri,4,0.209237,1040.0,0.020880
Nyctalus leisleri,0,0.021932,11.0,0.819202


In [91]:
sil_kmeans5 = sil_kmeans5.sort_values(['cluster', 'silhouette'])
sil_kmeans5.index = list(range(len(sil)))

plt.figure(figsize=(10, 8))

for cluster in set(sil_kmeans5['cluster']):
    plt.bar(sil_kmeans5[sil_kmeans5['cluster']==cluster].index,
         sil_kmeans5[sil_kmeans5['cluster']==cluster]['silhouette'],
         color = colourMap_5kmeans_dict[cluster], label='Cluster ' + str(cluster))

    
plt.title('Silhouette plot of 5 K-means clustering')

plt.legend()

plt.xlabel('Number of data point')
plt.ylabel('Silhouette coefficient')

#had to change this so that the legand was clearly visible.
plt.ylim(-1.4, 1.8)
plt.xlim(0,17)

(0, 17)

The clustering for k = 5 has many negative values which indicates that there is overlap between the clusters with some data points being closer to a neighbouring cluster then their own cluster. 

So it appears that k = 5 has not increased cluster cohesion and that the best clustering for the data is k = 4. 

# So the best clustering seems to be k = 4

How well does the clustering describe the actual bat population?

The clustering indicates that the population should fall into four broad categories. Those bats in the highest abundance grouping should be more likely to be found with other species. This assumes that the population is distributed relatively evenly.

To explore this I will examine the sites that have only multiple species present and then compare the abundance of species at those sites with the clustering results.

# Getting only those gridReference values that have more than one bat species present

In [92]:
grids_multiple_species = roost_short_df.groupby(['gridReference']).taxonName.nunique() > 1
grids_multiple_species.head()

gridReference
C5010    False
D1240    False
H0160    False
H1861     True
H2885     True
Name: taxonName, dtype: bool

Getting lists of values from the above grouping for later use.

In [93]:
#gridReferences
#grids_multiple_species.index

#values
#grids_multiple_species.values

gridRef_list = []
multiple_species = []

for x in grids_multiple_species.index:
    gridRef_list.append(x)
    
for x in grids_multiple_species.values:
    multiple_species.append(x)

Getting a list of sites with more than one species present.

In [94]:
temp_df = pd.DataFrame({'gridReference': grids_multiple_species.index,
                            'several_species': grids_multiple_species.values})
grid_multiple_true = temp_df[temp_df['several_species']==True]
grid_multiple_true.head()

,gridReference,several_species
3,H1861,True
4,H2885,True
6,H3524,True
8,H3723,True
29,J5570,True


Checking that one of the gridReferences does indeed have more than one species present.

In [95]:
roost_short_df[roost_short_df['gridReference']=='H1861'].sort_values(by='endDate').head(20)

,observationID,gridReference,precision,startDate,endDate,year,taxonName
20071,463198129,H1861,1km,1998-06-06,1998-06-06,1998,Pipistrellus pygmaeus
20072,463198130,H1861,1km,1998-06-15,1998-06-15,1998,Pipistrellus pygmaeus
20073,463198131,H1861,1km,1998-06-25,1998-06-25,1998,Pipistrellus pygmaeus
20076,463198134,H1861,1km,1999-06-20,1999-06-20,1999,Pipistrellus pygmaeus
20075,463198133,H1861,1km,1999-06-25,1999-06-25,1999,Pipistrellus pygmaeus
20074,463198132,H1861,1km,1999-12-06,1999-12-06,1999,Pipistrellus pygmaeus
20077,463198135,H1861,1km,2000-06-14,2000-06-14,2000,Pipistrellus pygmaeus
20078,463198136,H1861,1km,2000-06-21,2000-06-21,2000,Pipistrellus pygmaeus
20080,463198138,H1861,1km,2001-06-23,2001-06-23,2001,Pipistrellus pygmaeus
20079,463198137,H1861,1km,2001-12-06,2001-12-06,2001,Pipistrellus pygmaeus


It can be seen from the above that in 2004 there were more than one species of bat at the site H1861 at the same time.

Creating a new DataFrame to store the data of those gridRrferences that have multiple species present for later use.

In [96]:
multiple_species_df = pd.DataFrame(columns=('observationID', 'gridReference', 'precision', 'endDate', 'year', 'taxonName'))
multiple_species_df

,observationID,gridReference,precision,endDate,year,taxonName


Checking the number of gridReferences.

In [97]:
len(grid_multiple_true)

171

The code below iterates through gridReference values that have multiple bat species present then groups them into a single dataframe.

In [98]:
#This is quite slow but should take no longer than a minute.
#it took ~10 seconds.

i = 0
for x in grid_multiple_true.gridReference:
    for index, row in roost_df[roost_short_df['gridReference']==x].iterrows():
            multiple_species_df.loc[i] = [row.observationID,
                row.gridReference,
                row.precision,
                row.endDate,
                row.year,
                row.taxonName]
            i = 1 + i

Checking the results of the above.

In [99]:
multiple_species_df[multiple_species_df['gridReference']=='TL7273'].sort_values(by='year').tail(10)

#multiple_species_df.sort_values(by='endDate')

,observationID,gridReference,precision,endDate,year,taxonName
3845,463200672.0,TL7273,1km,2005-10-06,2005.0,Plecotus auritus
3839,463198561.0,TL7273,1km,2006-12-31,2006.0,Pipistrellus pipistrellus sensu lato
3827,463189822.0,TL7273,1km,2006-06-30,2006.0,Plecotus auritus
3826,463189821.0,TL7273,1km,2006-08-06,2006.0,Plecotus auritus
3829,463194987.0,TL7273,1km,2007-12-31,2007.0,Pipistrellus pipistrellus sensu lato
3830,463194989.0,TL7273,1km,2008-12-31,2008.0,Pipistrellus pipistrellus sensu lato
3831,463196055.0,TL7273,1km,2010-12-31,2010.0,Pipistrellus pipistrellus sensu lato
3832,463196056.0,TL7273,1km,2011-12-31,2011.0,Pipistrellus pipistrellus sensu lato
3828,463193804.0,TL7273,1km,2012-12-31,2012.0,Pipistrellus pipistrellus sensu lato
3844,463198586.0,TL7273,1km,2014-12-31,2014.0,Pipistrellus pipistrellus sensu lato


# Dealing with year values introducing unwanted results

Grouping the gridReferences that have multiple bats by year means that those site will on some years have only one bat species recorded.

The following is to filter out results that do not have more than one species present.

Getting boolean values for each site that indicate if multiple species are present at the same time.

In [100]:
multi_species_year_boolean = multiple_species_df.groupby(['gridReference', 'endDate', 'year'])['taxonName'].nunique() > 1
multi_species_year_boolean.head(20)

gridReference  endDate     year  
H1861          1998-06-06  1998.0    False
               1998-06-15  1998.0    False
               1998-06-25  1998.0    False
               1999-06-20  1999.0    False
               1999-06-25  1999.0    False
               1999-12-06  1999.0    False
               2000-06-14  2000.0    False
               2000-06-21  2000.0    False
               2001-06-23  2001.0    False
               2001-12-06  2001.0    False
               2001-12-31  2001.0    False
               2002-06-14  2002.0    False
               2002-06-21  2002.0    False
               2002-06-23  2002.0    False
               2003-06-13  2003.0    False
               2003-06-28  2003.0    False
               2003-12-31  2003.0    False
               2004-12-31  2004.0     True
               2005-06-25  2005.0    False
               2005-09-06  2005.0    False
Name: taxonName, dtype: bool

Getting a list of the boolean values.

In [101]:
multi_species_values = []
for x in multi_species_year_boolean.values:
    multi_species_values.append(x)

Grouping the data by bat species abundance.

In [102]:
multi_species_year_count = multiple_species_df.groupby(['gridReference', 'endDate', 'year', 'taxonName'])['taxonName'].count()
multi_species_year_count.to_frame().head()

taxonName
gridReference endDate    year   taxonName                       
H1861         1998-06-06 1998.0 Pipistrellus pygmaeus          1
              1998-06-15 1998.0 Pipistrellus pygmaeus          1
              1998-06-25 1998.0 Pipistrellus pygmaeus          1
              1999-06-20 1999.0 Pipistrellus pygmaeus          1
              1999-06-25 1999.0 Pipistrellus pygmaeus          1

In [103]:
multi_species_year_count_unstack = multi_species_year_count.unstack()
multi_species_year_count_unstack.head()

taxonName                        Chiroptera  Eptesicus serotinus  \
gridReference endDate    year                                      
H1861         1998-06-06 1998.0         NaN                  NaN   
              1998-06-15 1998.0         NaN                  NaN   
              1998-06-25 1998.0         NaN                  NaN   
              1999-06-20 1999.0         NaN                  NaN   
              1999-06-25 1999.0         NaN                  NaN   

taxonName                        Myotis brandtii  Myotis daubentonii  \
gridReference endDate    year                                          
H1861         1998-06-06 1998.0              NaN                 NaN   
              1998-06-15 1998.0              NaN                 NaN   
              1998-06-25 1998.0              NaN                 NaN   
              1999-06-20 1999.0              NaN                 NaN   
              1999-06-25 1999.0              NaN                 NaN   

taxonName                        Myotis mystacinus/brandtii  Myotis nattereri  \
gridReference endDate    year                                                   
H1861         1998-06-06 1998.0                         NaN               NaN   
              1998-06-15 1998.0                         NaN               NaN   
              1998-06-25 1998.0                         NaN               NaN   
              1999-06-20 1999.0                         NaN               NaN   
              1999-06-25 1999.0                         NaN               NaN   

taxonName                        Nyctalus leisleri  Nyctalus noctula  \
gridReference endDate    year                                          
H1861         1998-06-06 1998.0                NaN               NaN   
              1998-06-15 1998.0                NaN               NaN   
              1998-06-25 1998.0                NaN               NaN   
              1999-06-20 1999.0                NaN               NaN   
              1999-06-25 1999.0                NaN               NaN   

taxonName                        Pipistrellus pipistrellus sensu lato  \
gridReference endDate    year                                           
H1861         1998-06-06 1998.0                                   NaN   
              1998-06-15 1998.0                                   NaN   
              1998-06-25 1998.0                                   NaN   
              1999-06-20 1999.0                                   NaN   
              1999-06-25 1999.0                                   NaN   

taxonName                        Pipistrellus pipistrellus sensu stricto  \
gridReference endDate    year                                              
H1861         1998-06-06 1998.0                                      NaN   
              1998-06-15 1998.0                                      NaN   
              1998-06-25 1998.0                                      NaN   
              1999-06-20 1999.0                                      NaN   
              1999-06-25 1999.0                                      NaN   

taxonName                        Pipistrellus pygmaeus  Plecotus auritus  \
gridReference endDate    year                                              
H1861         1998-06-06 1998.0                    1.0               NaN   
              1998-06-15 1998.0                    1.0               NaN   
              1998-06-25 1998.0                    1.0               NaN   
              1999-06-20 1999.0                    1.0               NaN   
              1999-06-25 1999.0                    1.0               NaN   

taxonName                        Plecotus austriacus  \
gridReference endDate    year                          
H1861         1998-06-06 1998.0                  NaN   
              1998-06-15 1998.0                  NaN   
              1998-06-25 1998.0                  NaN   
              1999-06-20 1999.0                  NaN   
              1999-06-25 1999.0

Adding the boolean column and filliing zero values.

In [104]:
multi_species_year_count_unstack['multi_species'] = multi_species_values
multi_species_year_count_unstack.fillna(0, inplace=True)
multi_species_year_count_unstack.head()

taxonName                        Chiroptera  Eptesicus serotinus  \
gridReference endDate    year                                      
H1861         1998-06-06 1998.0         0.0                  0.0   
              1998-06-15 1998.0         0.0                  0.0   
              1998-06-25 1998.0         0.0                  0.0   
              1999-06-20 1999.0         0.0                  0.0   
              1999-06-25 1999.0         0.0                  0.0   

taxonName                        Myotis brandtii  Myotis daubentonii  \
gridReference endDate    year                                          
H1861         1998-06-06 1998.0              0.0                 0.0   
              1998-06-15 1998.0              0.0                 0.0   
              1998-06-25 1998.0              0.0                 0.0   
              1999-06-20 1999.0              0.0                 0.0   
              1999-06-25 1999.0              0.0                 0.0   

taxonName                        Myotis mystacinus/brandtii  Myotis nattereri  \
gridReference endDate    year                                                   
H1861         1998-06-06 1998.0                         0.0               0.0   
              1998-06-15 1998.0                         0.0               0.0   
              1998-06-25 1998.0                         0.0               0.0   
              1999-06-20 1999.0                         0.0               0.0   
              1999-06-25 1999.0                         0.0               0.0   

taxonName                        Nyctalus leisleri  Nyctalus noctula  \
gridReference endDate    year                                          
H1861         1998-06-06 1998.0                0.0               0.0   
              1998-06-15 1998.0                0.0               0.0   
              1998-06-25 1998.0                0.0               0.0   
              1999-06-20 1999.0                0.0               0.0   
              1999-06-25 1999.0                0.0               0.0   

taxonName                        Pipistrellus pipistrellus sensu lato  \
gridReference endDate    year                                           
H1861         1998-06-06 1998.0                                   0.0   
              1998-06-15 1998.0                                   0.0   
              1998-06-25 1998.0                                   0.0   
              1999-06-20 1999.0                                   0.0   
              1999-06-25 1999.0                                   0.0   

taxonName                        Pipistrellus pipistrellus sensu stricto  \
gridReference endDate    year                                              
H1861         1998-06-06 1998.0                                      0.0   
              1998-06-15 1998.0                                      0.0   
              1998-06-25 1998.0                                      0.0   
              1999-06-20 1999.0                                      0.0   
              1999-06-25 1999.0                                      0.0   

taxonName                        Pipistrellus pygmaeus  Plecotus auritus  \
gridReference endDate    year                                              
H1861         1998-06-06 1998.0                    1.0               0.0   
              1998-06-15 1998.0                    1.0               0.0   
              1998-06-25 1998.0                    1.0               0.0   
              1999-06-20 1999.0                    1.0               0.0   
              1999-06-25 1999.0                    1.0               0.0   

taxonName                        Plecotus austriacus  \
gridReference endDate    year                          
H1861         1998-06-06 1998.0                  0.0   
              1998-06-15 1998.0                  0.0   
              1998-06-25 1998.0                  0.0   
              1999-06-20 1999.0                  0.0   
              1999-06-25 1999.0

To get only the sites with multiple species present at the same time.

In [105]:
only_multi_species = multi_species_year_count_unstack[multi_species_year_count_unstack['multi_species']==True]
only_multi_species.tail(10)

taxonName                        Chiroptera  Eptesicus serotinus  \
gridReference endDate    year                                      
TR1652        2012-06-20 2012.0         0.0                  0.0   
              2012-07-26 2012.0         0.0                  0.0   
              2013-06-19 2013.0         0.0                  0.0   
              2013-07-07 2013.0         0.0                  0.0   
              2013-07-26 2013.0         0.0                  0.0   
              2013-10-06 2013.0         0.0                  0.0   
              2014-06-21 2014.0         0.0                  0.0   
              2014-07-06 2014.0         0.0                  0.0   
              2014-07-13 2014.0         0.0                  0.0   
              2014-07-26 2014.0         0.0                  0.0   

taxonName                        Myotis brandtii  Myotis daubentonii  \
gridReference endDate    year                                          
TR1652        2012-06-20 2012.0              0.0                 0.0   
              2012-07-26 2012.0              0.0                 0.0   
              2013-06-19 2013.0              0.0                 0.0   
              2013-07-07 2013.0              0.0                 0.0   
              2013-07-26 2013.0              0.0                 0.0   
              2013-10-06 2013.0              0.0                 0.0   
              2014-06-21 2014.0              0.0                 0.0   
              2014-07-06 2014.0              0.0                 0.0   
              2014-07-13 2014.0              0.0                 0.0   
              2014-07-26 2014.0              0.0                 0.0   

taxonName                        Myotis mystacinus/brandtii  Myotis nattereri  \
gridReference endDate    year                                                   
TR1652        2012-06-20 2012.0                         0.0               0.0   
              2012-07-26 2012.0                         0.0               0.0   
              2013-06-19 2013.0                         0.0               0.0   
              2013-07-07 2013.0                         0.0               0.0   
              2013-07-26 2013.0                         0.0               0.0   
              2013-10-06 2013.0                         0.0               0.0   
              2014-06-21 2014.0                         0.0               0.0   
              2014-07-06 2014.0                         0.0               0.0   
              2014-07-13 2014.0                         0.0               0.0   
              2014-07-26 2014.0                         0.0               0.0   

taxonName                        Nyctalus leisleri  Nyctalus noctula  \
gridReference endDate    year                                          
TR1652        2012-06-20 2012.0                0.0               0.0   
              2012-07-26 2012.0                0.0               0.0   
              2013-06-19 2013.0                0.0               0.0   
              2013-07-07 2013.0                0.0               0.0   
              2013-07-26 2013.0                0.0               0.0   
              2013-10-06 2013.0                0.0               0.0   
              2014-06-21 2014.0                0.0               0.0   
              2014-07-06 2014.0                0.0               0.0   
              2014-07-13 2014.0                0.0               0.0   
              2014-07-26 2014.0                0.0               0.0   

taxonName                        Pipistrellus pipistrellus sensu lato  \
gridReference endDate    year                                           
TR1652        2012-06-20 2012.0                                   0.0   
              2012-07-26 2012.0                                   0.0   
              2013-06-19 2013.0                                   0.0   
              2013-07-07 2013.0                                   0.0   
              2013-07-26 2013.0           

# The number of bats by species found cohabitating roost sites

Out of the 17 bat species in the dataset only 13 of them are involved in cohabitation and of these species only a minority of the abundance in the entire dataset are found cohabitating.

In [106]:
only_multi_species.sum().sort_values()

taxonName
Chiroptera                                   0.0
Nyctalus leisleri                            0.0
Plecotus austriacus                          3.0
Rhinolophus ferrumequinum                    7.0
Myotis mystacinus/brandtii                  12.0
Nyctalus noctula                            18.0
Myotis brandtii                             25.0
Myotis daubentonii                          28.0
Pipistrellus pipistrellus sensu lato        79.0
Myotis nattereri                            87.0
Rhinolophus hipposideros                    89.0
Eptesicus serotinus                         91.0
Pipistrellus pygmaeus                      171.0
Plecotus auritus                           246.0
Pipistrellus pipistrellus sensu stricto    277.0
multi_species                              539.0
dtype: float64

# Bats species that do not roost with any other species

These species of bats are not found cohabitating with any other bat species.

In [107]:
set(roost_short_df.taxonName.unique()) - set(only_multi_species.columns.unique())

{'Myotis bechsteinii', 'Myotis mystacinus'}

Additionally there is a zero abundance of Chiroptera and Nyctalus leisleri in the data for multiple speecies at the roost sites.

So from this it seems that the fllowing bats do not roost with other species of bats:

Myotis bechsteinii

Myotis mystacinus

Chiroptera

Nyctalus leisleri

# How does the abundance of sites with multiple species compare to the clusters?

In [108]:
multispecies_count_df = pd.DataFrame({'multispecies_count': only_multi_species.sum()})

In [109]:
multispecies_count_df.sum()

multispecies_count    1672.0
dtype: float64

In [110]:
result = pd.concat([multispecies_count_df, needed_for_later], axis=1, join='outer')

Renaming columns to aid readability.

In [111]:
result = result.rename(columns = {'multispecies_count': 'multi_species_abundance',
                               'sum': 'total_abundance'})
#result

In [112]:
#result.cluster

In [113]:
result.sort_values(by='multi_species_abundance')
#result.sort_values(by='sum')

,multi_species_abundance,cluster,standard_deviation,total_abundance,silhouette
Chiroptera,0.0,0.0,0.048548,54.0,0.933827
Nyctalus leisleri,0.0,0.0,0.021932,11.0,0.923419
Plecotus austriacus,3.0,0.0,0.014789,5.0,0.919823
Rhinolophus ferrumequinum,7.0,0.0,0.148263,514.0,0.460155
Myotis mystacinus/brandtii,12.0,0.0,0.037969,33.0,0.931358
Nyctalus noctula,18.0,0.0,0.056037,72.0,0.930832
Myotis brandtii,25.0,0.0,0.051168,60.0,0.933521
Myotis daubentonii,28.0,0.0,0.093279,160.0,0.881514
Pipistrellus pipistrellus sensu lato,79.0,1.0,0.360882,3478.0,0.607823
Myotis nattereri,87.0,3.0,0.209237,1040.0,0.487303


Reduced version of the above table to highlight results:

In [114]:
result2 = result[['multi_species_abundance', 'total_abundance', 'cluster']]
result2.sort_values(by='cluster')

,multi_species_abundance,total_abundance,cluster
Chiroptera,0.0,54.0,0.0
Plecotus austriacus,3.0,5.0,0.0
Nyctalus noctula,18.0,72.0,0.0
Rhinolophus ferrumequinum,7.0,514.0,0.0
Myotis mystacinus/brandtii,12.0,33.0,0.0
Nyctalus leisleri,0.0,11.0,0.0
Myotis daubentonii,28.0,160.0,0.0
Myotis brandtii,25.0,60.0,0.0
Myotis bechsteinii,NaN,22.0,0.0
Myotis mystacinus,NaN,97.0,0.0


# Interpretation of the table above, conclusion and confounding factors

The clustering seems to have correctly grouped the data at the low end with species not found to roost with others at all or in low numbers being in the same group.

The species most likely to be found roosting with others is: Pipistrellus pipistrellus sensu strict and this has been assigned to its own cluster so towards the high end of values the clustering also seems to be accurate.

The multi species abundance counts that could be described as falling in the middle seem to be outliers. 

The cluster containing the species Pipistrellus pipistrellus sensu lato, Rhinolophus hipposideros and Pipistrellus pygmaeus are all for relatively abundant bat species in the dataset and yet they are found in low abundance in terms of cohabitation.

Whereas the cluster containing Myotis nattereri, Eptesicus serotinus and Plecotus auritus has relatively high cohabitation counts compared to their total abundance.

As mentioned earlier some bat species don’t seem to be found cohabitating at all specifically, Myotis bechsteinii, Myotis mystacinus, Chiroptera and Nyctalus leisleri. With the notable exception of Chiroptera which has a total abundance of 54 and Myotis mystacinus with a total abundance of 97 these bat species have very low total abundances in the data so it is not surprising that they would not be found cohabitating.

These results are of limited value however as it is unsurprising that the most abundant bats would be more likely to be found cohabitating. 

Other limiting factors include the low number of data points used for clustering. Along with the k-means++ approach to selecting centroids for clustering as this tend to choose outlier values as the basis for clustering.

Another drawback of the analysis is that local and temporal factors are not accounted for. Bat species may be limited to certain areas for various reasons. The data observations are far from uniform and it seems likely that when observations were made and the quantity of those observations would have a large impact on any findings. It may also be that there are variations in roost counts due to the time the observations were made. But these are largely questions that cannot be accounted for in this dataset.

There is also the confounding factor of bat species population in general over time. Bats that are subject to extinction pressure or predatory cycles may be underrepresented in the data and these bats would in turn also be less likely to cohabitate.